This is a jupyter notebook version script for using UniKP 

from paper:
* https://www.nature.com/articles/s41467-023-44113-1

github source:
* https://github.com/Luo-SynBioLab/UniKP

After download the Prot-T5-XL-UniRef50 and the pre-trained UniKP_models('UniKP for Km.pkl','UniKP for Kcat.pkl','UniKP for kcat_Km.pkl') to UniKP folder and set up the environment, you can use the script for Km, Kcat Kcat/km prediction. 


In [9]:
import torch
from build_vocab import WordVocab
from pretrain_trfm import TrfmSeq2seq
from utils import split
# build_vocab, pretrain_trfm, utils packages are from SMILES Transformer
from transformers import T5EncoderModel, T5Tokenizer
# transformers package is from ProtTrans
import re
import gc
import numpy as np
import pandas as pd
import pickle
import math

#load kcat model
with open('UniKP/UniKP for kcat.pkl', "rb") as f:
    model_kcat= pickle.load(f)
# km
with open('UniKP/UniKP for Km.pkl', "rb") as f:
    model_km=pickle.load(f)
# kcat_km
with open('UniKP/UniKP for kcat_Km.pkl', "rb") as f:
    model_kcat_km=pickle.load(f)


def smiles_to_vec(Smiles):
    pad_index = 0
    unk_index = 1
    eos_index = 2
    sos_index = 3
    mask_index = 4
    vocab = WordVocab.load_vocab('vocab.pkl')
    def get_inputs(sm):
        seq_len = 220
        sm = sm.split()
        if len(sm)>218:
            print('SMILES is too long ({:d})'.format(len(sm)))
            sm = sm[:109]+sm[-109:]
        ids = [vocab.stoi.get(token, unk_index) for token in sm]
        ids = [sos_index] + ids + [eos_index]
        seg = [1]*len(ids)
        padding = [pad_index]*(seq_len - len(ids))
        ids.extend(padding), seg.extend(padding)
        return ids, seg
    def get_array(smiles):
        x_id, x_seg = [], []
        for sm in smiles:
            a,b = get_inputs(sm)
            x_id.append(a)
            x_seg.append(b)
        return torch.tensor(x_id), torch.tensor(x_seg)
    trfm = TrfmSeq2seq(len(vocab), 256, len(vocab), 4)
    trfm.load_state_dict(torch.load('trfm_12_23000.pkl'))
    trfm.eval()
    x_split = [split(sm) for sm in Smiles]
    xid, xseg = get_array(x_split)
    X = trfm.encode(torch.t(xid))
    return X

def Seq_to_vec(Sequence):
    for i in range(len(Sequence)):
        if len(Sequence[i]) > 1000:
            Sequence[i] = Sequence[i][:500] + Sequence[i][-500:]
    sequences_Example = []
    for i in range(len(Sequence)):
        zj = ''
        for j in range(len(Sequence[i]) - 1):
            zj += Sequence[i][j] + ' '
        zj += Sequence[i][-1]
        sequences_Example.append(zj)
    ###### you should place downloaded model into this directory.
    tokenizer = T5Tokenizer.from_pretrained("prot_t5_xl_uniref50", do_lower_case=False)
    model = T5EncoderModel.from_pretrained("prot_t5_xl_uniref50")
    gc.collect()
    print(torch.cuda.is_available())
    # 'cuda:0' if torch.cuda.is_available() else
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model = model.eval()
    features = []
    for i in range(len(sequences_Example)):
        print('For sequence ', str(i+1))
        sequences_Example_i = sequences_Example[i]
        sequences_Example_i = [re.sub(r"[UZOB]", "X", sequences_Example_i)]
        ids = tokenizer.batch_encode_plus(sequences_Example_i, add_special_tokens=True, padding=True)
        input_ids = torch.tensor(ids['input_ids']).to(device)
        attention_mask = torch.tensor(ids['attention_mask']).to(device)
        with torch.no_grad():
            embedding = model(input_ids=input_ids, attention_mask=attention_mask)
        embedding = embedding.last_hidden_state.cpu().numpy()
        for seq_num in range(len(embedding)):
            seq_len = (attention_mask[seq_num] == 1).sum()
            seq_emd = embedding[seq_num][:seq_len - 1]
            features.append(seq_emd)
    features_normalize = np.zeros([len(features), len(features[0][0])], dtype=float)
    for i in range(len(features)):
        for k in range(len(features[0][0])):
            for j in range(len(features[i])):
                features_normalize[i][k] += features[i][j][k]
            features_normalize[i][k] /= len(features[i])
    return features_normalize


    
def site_saturated_mutagenesis(sequences:str,Smiles:str,site:int,model_kcat,model_km,model_kcat_km,file_name):
    pos=site-1
    AA=['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
    seq=list(sequences)
    smiles_vec=smiles_to_vec([Smiles])
    res_dict={'Name':[],'sequences': [], 'Smiles': [], 'Kcat': [],'Km':[],'kcat_km':[]}
    original=seq[pos]
    for i in AA:
        name=f'{original}{site}{i}'
        seq[pos]=i
        mutated_seq=''.join(seq)
        seq_vec=Seq_to_vec([mutated_seq])
        fused_vector=np.concatenate((smiles_vec,seq_vec),axis=1)
        predicted_kcat=model_kcat.predict(fused_vector)
        predicted_km=model_km.predict(fused_vector)
        predicted_kcat_km=model_kcat_km.predict(fused_vector)
        res_dict['Name'].append(name)
        res_dict['sequences'].append(mutated_seq)
        res_dict['Smiles'].append(Smiles)
        res_dict['Kcat'].append(10**predicted_kcat[0])
        res_dict['Km'].append(10**predicted_km[0])
        res_dict['kcat_km'].append(10**predicted_kcat_km[0])
    res=pd.DataFrame(res_dict)
    res.to_excel(f'{file_name}_predict_result.xlsx')
    res.to_csv(f'{file_name}_predict_result.csv')
    print('Successfully write the table!')
    return res

def predict_kinetics(sequences:str,Smiles:str,file_name):
    res_dict={'Name':[],'sequences': [], 'Smiles': [], 'Kcat': [],'Km':[],'kcat_km':[]}
    seq_vec = Seq_to_vec([sequences])
    smiles_vec=smiles_to_vec([Smiles])
    fused_vector=np.concatenate((smiles_vec,seq_vec),axis=1)
    predicted_kcat=model_kcat.predict(fused_vector)
    predicted_km=model_km.predict(fused_vector)
    predicted_kcat_km=model_kcat_km.predict(fused_vector)
    res_dict['Name'].append(file_name)
    res_dict['sequences'].append(sequences)
    res_dict['Smiles'].append(Smiles)
    res_dict['Kcat'].append(10**predicted_kcat[0])
    res_dict['Km'].append(10**predicted_km[0])
    res_dict['kcat_km'].append(10**predicted_kcat_km[0])
    res=pd.DataFrame(res_dict)
    res.to_excel(f'{file_name}_predict_result.xlsx')
    res.to_csv(f'{file_name}_predict_result.csv')
    print('Successfully write the table!')
    return res

#### Input your data here:

In [10]:
sequences="MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGNANELNASYMADGYA\
RTKKAAAFLTTFGVGELSAVNGLAGSYAENLPVVEIVGSPTSKVQNEGKFVHHTLADGDF\
KHFMKMHEPVTAARTLLTAENATVEIDRVLSALLKERKPVYINLPVDVAAAKAEKPSLPL\
KKENSTSNTSDQEILNKIQESLKNAKKPIVITGHEIISFGLEKTVTQFISKTKLPITTLN\
FGKSSVDEALPSFLGIYNGTLSEPNLKEFVESADFILMLGVKLTDSSTGAFTHHLNENKM\
ISLNIDEGKIFNERIQNFDFESLISSLLDLSEIEYKGKYIDKKQEDFVPSNALLSQDRLW\
QAVENLTQSNETIVAEQGTSFFGASSIFLKSKSHFIGQPLWGSIGYTFPAALGSQIADKE\
SRHLLFIGDGSLQLTVQELGLAIREKINPICFIINNDGYTVEREIHGPNQSYNDIPMWNY\
SKLPESFGATEDRVVSKIVRTENEFVSVMKEAQADPNRMYWIELILAKEGAPKVLKKMGK\
LFAEQNKS"
Smiles = 'CCCCC(=O)C(=O)[O-]'
position=461

#### kcat km prediction

In [11]:
predict_kinetics(sequences,Smiles,'2KC')

Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

True
For sequence  1
Successfully write the table!


,Name,sequences,Smiles,Kcat,Km,kcat_km
0,2KC,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],6.981965,0.530508,5.341446


#### site-saturated mutagenesis

In [3]:
result=site_saturated_mutagenesis(sequences, Smiles,position,model_kcat,model_km,model_kcat_km,'2KC')
result

Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

True
For sequence  1


Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

True
For sequence  1


Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

...

Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

True
For sequence  1


Some weights of the model checkpoint at prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.14.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.o.weight', 'decoder.block.15.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.14.layer.2.DenseReluDense.wi.weight', 'decoder.block.14.layer.0.SelfAttention.v.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.2.layer_norm.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.8.layer.1.EncDecAttention.v.weight', 'decoder.block.16.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.1.EncDecAttention.k.weight', 'decoder.block.12.layer.0.layer_norm.weight

True
For sequence  1
Successfully write the table!


,Name,sequences,Smiles,Kcat,Km,kcat_km
0,V461A,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],5.708540,0.553294,5.364692
1,V461R,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],4.907474,0.631610,4.865356
2,V461N,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],5.974817,0.608845,6.375361
3,V461D,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],4.649652,0.616269,5.572117
4,V461C,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],4.058017,0.614726,5.155130
5,V461Q,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],5.322117,0.608845,5.281389
6,V461E,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],5.125776,0.608845,5.623400
7,V461G,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],4.987110,0.617082,5.636923
8,V461H,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],5.214122,0.609540,5.002969
9,V461I,MYTVGDYLLDRLHELGIEEIFGVPGDYNLQFLDQIISHKDMKWVGN...,CCCCC(=O)C(=O)[O-],7.469082,0.583772,5.633235
